In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
datasets = [1, 2, 3, 4 , 5]
exps = [0.1, 0.25, 0.5, 0.75, 0.9]

In [7]:
for d in datasets:
    for e in exps:
        df_enc = pd.read_csv(f'../data/Athey_v1/500/data_train_enc_{str(e)}_{str(d)}.csv')
        df = pd.read_csv(f'../data/Athey_v1/500/data_train_{str(e)}_{str(d)}.csv')
        for i in ['lasso0', 'lasso1', 'linear0', 'linear1']:
            df[i] = df_enc[i]
        df.to_csv(f'../data/Athey_v1/500/data_train_{str(e)}_{str(d)}.csv', index=False)

In [8]:
df

,V1,V2,y0,y1,t,prob_t,y,prob_t_pred_log,prob_t_pred_tree,lasso0,lasso1,linear0,linear1
0,-0.037514,0.016442,0.563711,0.555081,0,0.9,0.563711,0.548764,0.924603,0.530176,0.618931,0.555297,0.541046
1,-1.574604,-0.082065,0.448161,0.353234,0,0.9,0.448161,0.983691,0.924603,0.520338,0.613203,0.472355,0.359600
2,-0.485968,1.464694,0.780444,0.691904,0,0.9,0.780444,0.796030,0.924603,0.674818,0.703141,0.777736,0.706431
3,0.465186,1.066794,0.751054,0.744486,1,0.9,0.744486,0.743163,0.903226,0.635078,0.680005,0.752438,0.750884
4,-0.904098,-1.857091,0.220040,0.181238,0,0.9,0.220040,0.913994,0.924603,0.343060,0.509991,0.204955,0.170021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.255153,-1.468567,0.305165,0.389843,1,0.9,0.389843,0.639946,0.903226,0.381863,0.532583,0.319982,0.353313
496,-0.664306,-0.174358,0.478531,0.434954,0,0.9,0.478531,0.856296,0.924603,0.511120,0.607836,0.496322,0.444790
497,0.472618,0.440170,0.683276,0.635843,1,0.9,0.635843,0.748911,0.903226,0.572495,0.643569,0.648122,0.659065
498,-0.706736,0.735993,0.642301,0.593743,0,0.9,0.642301,0.870942,0.924603,0.602040,0.660770,0.646503,0.574748


In [7]:
import pandas as pd
import math
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import collections
import sklearn.metrics as skm

df = pd.read_csv('../raw.csv')
seed = 109
# 13, 1, 5, 19, 109
path = 'seed5/'

# VCORC1 imputation rules
impute = df['VKORC1 rs9923231'].isna()

for index, row in df.iterrows():
    #print(type(row['VKORC1 QC genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T']))
    # determine if the rs9923231 row is missing
    if impute[index]:
        # check the QC column?
        if row['Race (OMB)'] in ['White', 'Asian']:
            if row['VKORC1 rs2359612'] == 'C/C':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'G/G'
            if row['VKORC1 rs2359612'] == 'T/T':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/A'
            if row['VKORC1 rs2359612'] == 'C/T':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/G'
            if row['VKORC1 rs8050894'] == 'G/G':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'G/G'
            if row['VKORC1 rs8050894'] == 'C/C':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/A'
            if row['VKORC1 rs8050894'] == 'C/G':
                df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/G'
        if row['VKORC1 rs9934438'] == 'C/C':
            df.loc[df.index[index], 'VKORC1 rs9923231'] = 'G/G'
        if row['VKORC1 rs9934438'] == 'T/T':
            df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/A'
        if row['VKORC1 rs9934438'] == 'C/T':
            df.loc[df.index[index], 'VKORC1 rs9923231'] = 'A/G'


# --- label binarizer for VKORC1 ---
# only keep columns A/G, A/A, unknown (there are 126 missing still)
def vkorc1_ag(s):
    if s['VKORC1 rs9923231'] == 'A/G':
        return 1
    else:
        return 0
df['VKORC1 A/G'] = df.apply(vkorc1_ag, axis=1)

def vkorc1_aa(s):
    if s['VKORC1 rs9923231'] == 'A/A':
        return 1
    else:
        return 0
df['VKORC1 A/A'] = df.apply(vkorc1_aa, axis=1)

def vkorc1_na(s):
    if pd.isnull(s['VKORC1 rs9923231']):
        return 1
    else:
        return 0
df['VKORC1 Missing'] = df.apply(vkorc1_na, axis=1)

# ---- DELETE MISSING ENTRIES THAT CANNOT BE IMPUTED
# ---- AGE ----
# discard all entries with missing age (42)
df = df[df['Age'].notna()]

# ----- HEIGHT AND WEIGHT -----
# Drop all missing values of height and weight
df = df[df['Height (cm)'].notna()]
df = df[df['Weight (kg)'].notna()] # 4490

# TREATMENT
df = df[df['Therapeutic Dose of Warfarin'].notna()] # 4386


## ------ RACE ------
lb = LabelBinarizer()
race = lb.fit_transform(df['Race (OMB)'])
classes = lb.classes_
race = pd.DataFrame(race, columns=classes)
# only keep Asian, Black/African American, Unknown
## --- POTENTIAL PROBLEM: most of the people are white --> missing group
race = race[['Asian', 'Black or African American', 'Unknown']].rename(columns={'Unknown': 'Unknown Race'}) # must combine this with full dataset

# --- cyp2C9 genotype ---
#print(df['Cyp2C9 genotypes'].value_counts())
#print(df['Cyp2C9 genotypes'].isna().sum())
df['Cyp2C9 genotypes'] = df['Cyp2C9 genotypes'].fillna('Unknown Cyp2C9')
lb = LabelBinarizer()
cyp2c9 = lb.fit_transform(df['Cyp2C9 genotypes'])
classes = lb.classes_
cyp2c9 = pd.DataFrame(cyp2c9, columns=classes)
cyp2c9 = cyp2c9[['*1/*1', '*1/*3', '*2/*2', '*2/*3', '*3/*3', 'Unknown Cyp2C9']] # need to add this back to all dataframe

# --- ENZYME INDUCER ---
# 1 if patient taking carbamazepine (tegretol), phenytoin (dilantin), rifampin or rifampicin, otherwise zero
def enzyme(s):
    if (s['Carbamazepine (Tegretol)'] == 1) or (s['Phenytoin (Dilantin)'] == 1) or (s['Rifampin or Rifampicin'] == 1):
        return 1
    else:
        return 0

df['Enzyme Inducer'] = df.apply(enzyme, axis=1)
#print(df['Enzyme Inducer'].value_counts())

# --- AMIODARONE STATUS ---
# 1 if patient taking amiodarone (cordarone), otherwise zero
# there are 1518 nan values
df['Amiodarone (Cordarone)'] = df['Amiodarone (Cordarone)'].fillna(0)
#print(df['Amiodarone (Cordarone)'].value_counts())


def stratify(m, columns):
    for i in range(len(m)):
        if m[i][0] == 1:
            m[i] = [1, 1, 1, 1, 1]
        elif m[i][1] == 1:
            m[i] = [0, 1, 1, 1, 1]
        elif m[i][2] == 1:
            m[i] = [0, 0, 1, 1, 1]
        elif m[i][3] == 1:
            m[i] = [0, 0, 0, 1, 1]
        elif m[i][4] == 1:
            m[i] = [0, 0, 0, 0, 1]
    m = pd.DataFrame(data=m, columns=columns)
    return m

# ---- DISCRETIZING AGE, HEIGHT, WEIGHT ----
### AGE
df['Age'] = df['Age'].astype(str).str[0].astype(int)

bins = [0, 2, 4, 6, 7, 9]
labels = [1, 2, 3, 4, 5]
df['age_bins'] = pd.cut(df['Age'], bins=bins, labels=labels)

lb = LabelBinarizer()
age = lb.fit_transform(df['age_bins'])
age = stratify(age, ['Age1-2', 'Age3-4', 'Age5-6', 'Age7', 'Age8-9'])

### HEIGHT
buffer = df['Height (cm)'].sort_values().to_numpy()
inc = int(len(buffer)/5)
bins = [buffer[0]-1, buffer[inc], buffer[inc*2], buffer[inc*3], buffer[inc*4], buffer[-1]]
#bins = [124.97, 158.0, 165.1, 170.94, 178.0, 202.0]
labels = [1, 2, 3, 4, 5]
df['height_bins'] = pd.cut(df['Height (cm)'], bins=bins, labels=labels)

lb = LabelBinarizer()
height = lb.fit_transform(df['height_bins'])
height = stratify(height, ['Height1', 'Height2', 'Height3', 'Height4', 'Height5'])

### WEIGHT
buffer = df['Weight (kg)'].sort_values().to_numpy()
inc = int(len(buffer)/5)
bins = [buffer[0]-1, buffer[inc], buffer[inc*2], buffer[inc*3], buffer[inc*4], buffer[-1]]
#bins = [124.97, 158.0, 165.1, 170.94, 178.0, 202.0]
labels = [1, 2, 3, 4, 5]
df['weight_bins'] = pd.cut(df['Weight (kg)'], bins=bins, labels=labels)

lb = LabelBinarizer()
weight = lb.fit_transform(df['weight_bins'])
weight = stratify(weight, ['Weight1', 'Weight2', 'Weight3', 'Weight4', 'Weight5'])


# combining everything together
rest = df[['Enzyme Inducer', 'Amiodarone (Cordarone)', 'VKORC1 A/G', 'VKORC1 A/A', 'VKORC1 Missing', 'Therapeutic Dose of Warfarin']]
rest = rest.reset_index().drop(columns=['index'])
# COMPILE EVERYTHING (bucketized)

# csv not bucketized
rest2 = df[['Age', 'Height (cm)', 'Weight (kg)']]
rest2 = rest2.rename(columns={'Height (cm)': 'Height', 'Weight (kg)': 'Weight'}).reset_index().drop(columns=['index'])
df = pd.concat([rest2, race, cyp2c9, rest], axis=1)


# --- DISCRETIZING OUTCOME AND TREATMENT ---
# Drop all missing values of no optimal doe

bins = [0, 21, 49, 350]
labels = [0, 1, 2]

# transform t_ideal from the formula
# df['t_ideal'] = pd.cut(df['Therapeutic Dose of Warfarin'], bins=bins, labels=labels)

cols = ['Age', 'Height', 'Weight', 'VKORC1 A/G', 'VKORC1 A/A', 'VKORC1 Missing', '*1/*1', '*1/*3', '*2/*2',
        '*2/*3', '*3/*3', 'Unknown Cyp2C9', 'Asian', 'Black or African American', 'Unknown Race',
        'Enzyme Inducer', 'Amiodarone (Cordarone)']

coeffs = [-0.2614, 0.0087, 0.0128, -0.8677, -1.6974, -0.4854, -0.5211, -0.9357, -1.0616, -1.9206, -2.3312, -0.2188,
          -0.1092, -0.2760, -0.1032, 1.1816, -0.5503]

const = 5.6044
df['t_ideal'] = const
for i in range(len(cols)):
    df['t_ideal'] += coeffs[i] * df[cols[i]]


# Add noise
noise = np.random.normal(0, 0.2, size=len(df))

df['t_ideal_noise'] = df['t_ideal'] + noise

df['t_ideal'] = df['t_ideal'] * df['t_ideal']
df['t_ideal_noise'] = df['t_ideal_noise'] * df['t_ideal_noise']
#print(skm.r2_score(df['Therapeutic Dose of Warfarin'], df['t_ideal']))
df['t_ideal'] = pd.cut(df['t_ideal'], bins=bins, labels=labels)
df['t_ideal_noise'] = pd.cut(df['t_ideal_noise'], bins=bins, labels=labels)

In [8]:
df['t_ideal'].value_counts()

1    2784
0    1487
2     115
Name: t_ideal, dtype: int64

In [9]:
for i in df['t_ideal'].unique():
    print(i)
    print(len(df[df['t_ideal'] == i])/len(df))

1
0.6347469220246238
2
0.026219790241678066
0
0.3390332877336981
